<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/PH_NER_CreateDataset_ChildParent_UNFINISHED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Declare whether you are on Colab or local
colab = True

In [ ]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')

  #set path to data in Google Drive
  data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"
  data_path_group = data_path[:-10]+"Topic 1/Data_Team1/" # create new data path to access files created by Team1

  !pip install duckdb

else:
    data_path_group = "../data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 24.2 MB/s 


In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import duckdb
import sqlalchemy

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Task 1: New entity recognition  

## Load data

### Ontology

In [ ]:
NER_ontology_train_df = pd.read_csv(data_path_group+"NER/NER_ontology_train.csv", index_col=0)
NER_ontology_val_df = pd.read_csv(data_path_group+"NER/NER_ontology_val.csv", index_col=0)
NER_ontology_test_df = pd.read_csv(data_path_group+"NER/NER_ontology_test.csv", index_col=0)

In [ ]:
NER_ontology_train_df.head(5)

,entity_id,category,label,synonym
0,IS topic,domain specific entity,TOPIC,IS topic
1,IS topic,domain specific entity,TOPIC,IS topics
2,IS sourcing,domain specific entity,TOPIC,IS / IT sourcing
3,IS sourcing,domain specific entity,TOPIC,tool sourcing
4,IS sourcing,domain specific entity,TOPIC,software sourcings


#### Noun phrases

In [ ]:
NER_entities_train_df = pd.read_csv(data_path_group+"NER/NER_entities_train.csv", index_col=0)
NER_entities_val_df = pd.read_csv(data_path_group+"NER/NER_entities_val.csv", index_col=0)
NER_entities_test_df = pd.read_csv(data_path_group+"NER/NER_entities_test.csv", index_col=0)

In [ ]:
NER_entities_train_df.tail(3)

,article_id,sentence_id,ent_id,noun_phrases,true_ent_id
13172374,4994,4994_204_223,programming language,programming languages,programming language
13172375,4994,4994_204_223,programming language,globalization,0
13172376,4994,4994_204_223,programming language,IS.ABSTRACT,0


#### Entities table from DuckDB

In [ ]:
con = duckdb.connect(database=data_path+'/Extracted Information of Corpus/isrecon.duckdb', read_only=False)

In [ ]:
# Load first article as df
entities_df = entities = con.execute("""
                       SELECT article_id, sentence_id, ent_id, label, level_1, level_2, level_3, level_4, level_5, level_6, level_7, level_8, level_9, level_10,
                       FROM entities e
                       """).fetchdf()

entities_df.shape

(1797740, 14)

In [ ]:
entities_df.head(3)

,article_id,sentence_id,ent_id,label,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10
0,188,188_3753_3772,videotex,TECHNOLOGY,domain specific entity,IS technology,information technology infrastructure,communication service infrastructure,videotex,NaN,NaN,NaN,NaN,NaN
1,188,188_3828_3848,competitive advantage,TOPIC,domain specific entity,IS topic,IS evaluation,evaluation criteria,IS evaluation criteria,IS impact,organizational impact,competitive advantage,NaN,NaN
2,188,188_3848_3883,government,SECTOR,domain specific entity,economic sector,government,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add parent id to each entity

In [ ]:
# Lets drop all the duplicates since we only care about the parent child node relatinship between the different entities
entities_uniq_df = entities_df.drop_duplicates(subset="ent_id").drop(columns=["article_id", "sentence_id"])
entities_uniq_df.shape

(2130, 12)

In [ ]:
entities_uniq_df.head(2)

,ent_id,label,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10
0,videotex,TECHNOLOGY,domain specific entity,IS technology,information technology infrastructure,communication service infrastructure,videotex,NaN,NaN,NaN,NaN,NaN
1,competitive advantage,TOPIC,domain specific entity,IS topic,IS evaluation,evaluation criteria,IS evaluation criteria,IS impact,organizational impact,competitive advantage,NaN,NaN


In [ ]:
# Define function to get last index which is not NaN => The level of the entity within the ontology
def get_last(a):
  '''Get last index of list which is not NaN'''
  for i, e in enumerate(reversed(a)):
    if e is not np.nan:
      return len(a) - i -1
  return -1

In [ ]:
# Function to take a dataframe and create dictonary of level of entity, level of parent entity and name of parent entity
def get_idx_par_ent(df):
  '''Get level of entity, level of parent entity and name of parent entity'''

  col_names = ['ent_id', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'level_6', 'level_7', 'level_8', 'level_9', 'level_10']
  ent_lvls = df[col_names].values.flatten().tolist() 
  lvl_ent = get_last(ent_lvls)
  lvl_par_ent = lvl_ent-1
  par_ent = ent_lvls[lvl_par_ent]

  return {"ent_id":ent_lvls[0], "lvl_ent":lvl_ent, "lvl_par_ent":lvl_par_ent, "par_ent":par_ent}

In [ ]:
entities_par_df = entities_uniq_df.apply(lambda x: get_idx_par_ent(x), axis=1, result_type='expand')

In [ ]:
entities_par_df.head(2)

,ent_id,lvl_ent,lvl_par_ent,par_ent
0,videotex,5,4,communication service infrastructure
1,competitive advantage,8,7,organizational impact


In [ ]:
#entities_par_df.to_csv(data_path_group+"NER/Parent_Dataset/NER_entities_parents_cmpl.csv")

## Create binary classification training dataset for entity and parent entitiy

In [ ]:
# Add training label column
entities_par_df["is_parent"] = 1

In [ ]:
entities_mrg = entities_uniq_df.merge(entities_par_df, how="inner", on="ent_id")

Now our dataset only consists of positive examples, but we should also provide the model with some negative examples. The question is here what would be a "good" negative example, so that the model learns the similarity between the parent and the child node ...

We came up with different ideas for negative examples: 
1. Child entity vs. other child entity from the same parent entity
2. Child entity vs. other parent entity within the same branch as parent entity
3. Child entity vs. other parent entity outside of the parent entity's branch (but still in the same category)

We decided to use option 3, because after training the transformer still should put the different child entities from the same parent close together as well as other parents entities which are on the same level and in the same branch.

In [ ]:
# Function to take a dataframe and create dictonary of level of entity, level of parent entity and name of parent entity
def get_idx_grandpar_ent(df):
  '''Get level of entity, level of parent entity and name of parent entity'''

  col_names = ['ent_id', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'level_6', 'level_7', 'level_8', 'level_9', 'level_10']
  ent_lvls = df[col_names].values.flatten().tolist() 
  lvl_grpar_ent = get_last(ent_lvls)-2
  grpar_ent = ent_lvls[lvl_grpar_ent]

  return {"grpar_ent":grpar_ent, "lvl_grandpar_ent":lvl_grpar_ent}

In [ ]:
entities_mrg.head(2)

,ent_id,label,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10,lvl_ent,lvl_par_ent,par_ent,is_parent
0,videotex,TECHNOLOGY,domain specific entity,IS technology,information technology infrastructure,communication service infrastructure,videotex,NaN,NaN,NaN,NaN,NaN,5,4,communication service infrastructure,1
1,competitive advantage,TOPIC,domain specific entity,IS topic,IS evaluation,evaluation criteria,IS evaluation criteria,IS impact,organizational impact,competitive advantage,NaN,NaN,8,7,organizational impact,1


In [ ]:
#entities_tmp[["grpar_ent", "lvl_grandpar_ent"]]
entities_mrg[["grpar_ent", "lvl_grandpar_ent"]] = entities_mrg.apply(lambda x: get_idx_grandpar_ent(x), axis=1, result_type='expand')

In [ ]:
entities_mrg.head(2)

,ent_id,label,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10,lvl_ent,lvl_par_ent,par_ent,is_parent,grpar_ent,lvl_grandpar_ent
0,videotex,TECHNOLOGY,domain specific entity,IS technology,information technology infrastructure,communication service infrastructure,videotex,NaN,NaN,NaN,NaN,NaN,5,4,communication service infrastructure,1,information technology infrastructure,3
1,competitive advantage,TOPIC,domain specific entity,IS topic,IS evaluation,evaluation criteria,IS evaluation criteria,IS impact,organizational impact,competitive advantage,NaN,NaN,8,7,organizational impact,1,IS impact,6


In [ ]:
# Replace entity name for level 0 with np.nan
entities_mrg.loc[entities_mrg.lvl_grandpar_ent==0,"grpar_ent"] = np.nan

In [ ]:
entities_mrg[entities_mrg.lvl_grandpar_ent==0]

,ent_id,label,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10,lvl_ent,lvl_par_ent,par_ent,is_parent,grpar_ent,lvl_grandpar_ent
24,IS technology,TECHNOLOGY,domain specific entity,IS technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,domain specific entity,1,NaN,0
54,level of analysis,LEVEL,theoretical entity,level of analysis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,theoretical entity,1,NaN,0
121,theory,THEORY,theoretical entity,theory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,theoretical entity,1,NaN,0
135,sampling,SAMPLING,methodological entity,sampling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,methodological entity,1,NaN,0
184,data analysis method,ANALYSIS_METHOD,methodological entity,data analysis method,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,methodological entity,1,NaN,0
476,research paradigm,PARADIGM,theoretical entity,research paradigm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,theoretical entity,1,NaN,0
1075,IS topic,TOPIC,domain specific entity,IS topic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,domain specific entity,1,NaN,0
1352,economic sector,SECTOR,domain specific entity,economic sector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,domain specific entity,1,NaN,0
1440,validity,VALIDITY,methodological entity,validity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,methodological entity,1,NaN,0
1933,statistical metric,METRIC,methodological entity,statistical metric,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,methodological entity,1,NaN,0
